In [1]:
import torch
import torchvision
import os
import numpy as np
import torchvision.transforms as transforms
root = './data'
if not os.path.exists(root):
    os.mkdir(root)

In [8]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(1.0,))])
trainset = torchvision.datasets.MNIST(root=root, train=False,transform=transform, download=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=500,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.MNIST(root=root, train=False, transform=transform, download=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=500,
                                         shuffle=False, num_workers=2)
classes = ('0','1','2','3','4','5','6','7','8','9')
#print(len(trainset))
#print(len(testset))

In [3]:
import matplotlib.pyplot as plt
import numpy as np
def imshow(img):
    img = img + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

dataiter = iter(trainloader)
images, labels = dataiter.next()
print(labels)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
#print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

tensor([7, 4, 0, 9, 1, 3, 6, 1, 7, 0, 2, 2, 1, 5, 2, 7, 0, 8, 8, 9, 3, 6, 2, 4,
        7, 4, 2, 1, 9, 9, 0, 9, 4, 9, 3, 3, 3, 8, 7, 9, 2, 2, 8, 3, 7, 6, 1, 1,
        2, 4, 0, 8, 3, 4, 3, 5, 4, 0, 2, 7, 7, 5, 9, 4, 1, 6, 3, 5, 1, 3, 6, 0,
        8, 8, 4, 3, 8, 8, 2, 7, 4, 2, 3, 0, 1, 3, 0, 3, 3, 5, 2, 5, 4, 0, 4, 6,
        7, 9, 1, 1, 4, 1, 0, 0, 2, 5, 9, 6, 1, 8, 5, 7, 5, 8, 3, 9, 8, 0, 0, 0,
        1, 0, 2, 3, 2, 9, 4, 4, 1, 9, 2, 6, 3, 6, 1, 3, 1, 6, 3, 3, 9, 3, 3, 1,
        9, 9, 4, 8, 4, 2, 5, 0, 1, 8, 8, 0, 8, 6, 1, 2, 9, 3, 6, 5, 8, 9, 0, 2,
        2, 6, 9, 0, 3, 3, 4, 9, 5, 8, 1, 4, 3, 4, 3, 5, 3, 8, 3, 3, 3, 7, 1, 1,
        5, 7, 0, 5, 7, 4, 7, 9, 8, 6, 9, 0, 2, 1, 8, 0, 3, 8, 5, 4, 2, 7, 6, 1,
        8, 7, 1, 3, 5, 2, 0, 3, 4, 5, 1, 6, 1, 6, 6, 7, 5, 7, 5, 6, 3, 3, 3, 4,
        2, 7, 4, 8, 2, 8, 2, 2, 6, 8, 1, 2, 2, 6, 3, 9, 4, 0, 9, 1, 7, 9, 2, 1,
        2, 1, 0, 0, 9, 4, 7, 2, 3, 4, 5, 9, 0, 1, 0, 9, 6, 1, 8, 6, 0, 4, 7, 9,
        1, 6, 3, 0, 1, 6, 1, 7, 3, 8, 5,

In [9]:
import torch.nn as nn
import torch.nn.functional as F
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def name(self):
        return "LeNet"
        

In [10]:
import torch.optim as optim
model = LeNet()
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [11]:
for epoch in range(10):
    # trainning
    ave_loss = 0
    for i,data in enumerate(trainloader,0):
        inputs, label = data
        optimizer.zero_grad()
        out = model(inputs)
        loss = criterion(out,label)
        loss.backward()
        optimizer.step()
        ave_loss+= loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, ave_loss / 200))
            ave_loss = 0.0
print("Finished Training")


Finished Training


In [12]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(total)
print(correct)
print(100 * correct / total)

10000
9949
99.49
